Needed setup  

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

%ls
%cd drive/MyDrive/
%cd Colab_Notebooks/faces_datasets/
# move to correct folder

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
egohands_dataset_to_csv.py  hand_over_face/  images/
[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive/Colab Notebooks/faces_datasets
[Errno 2] No such file or directory: 'Colab_Notebooks/faces_datasets/'
/content/drive/MyDrive/Colab Notebooks/faces_datasets


In [ ]:
# copied from https://towardsdatascience.com/train-an-object-detector-using-tensorflow-2-object-detection-api-in-2021-a4fed450d1b9
# use script to download and "setup" egohands
!git clone https://github.com/aalpatya/detect_hands.git
!cp detect_hands/egohands_dataset_to_csv.py .
!python egohands_dataset_to_csv.py

In [ ]:
# remove useless files
!rm -dr detect_hands/
!rm -f egohands_data.zip
!rm -dr egohands/

rm: cannot remove 'egohands/': No such file or directory


In [ ]:
#unpack the hand_pver_face dataset 
!tar -xvzf hand_over_face_corrected.tar.gz

In [ ]:
#remove dataset file
!rm -f hand_over_face_corrected.tar.gz


## Here we start with the actual code


In [ ]:
# basic imports
import torchvision
import numpy as np
import torch
from PIL import Image
import os
os.getcwd()


'/content'

In [ ]:
# import Mask RCNN model 
# https://pytorch.org/vision/master/models/mask_rcnn.html

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()


### loading one image and visualizing the model
very basic

In [ ]:
import torchvision.transforms as transforms
imsize = 256

loader = transforms.Compose([transforms.ToTensor()])

def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

model(image_loader("images/train/CARDS_LIVINGROOM_T_B_frame_0001.jpg"))

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

import cv2, random
import matplotlib.pyplot as plt

image_path = "images/train/CARDS_LIVINGROOM_T_B_frame_0001.jpg"

def get_prediction(img_path, threshold):
  img = Image.open(img_path)
  transform = transforms.Compose([transforms.ToTensor()])
  img = transform(img)
  pred = model([img])
  pred_score = list(pred[0]['scores'].detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
  masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
  masks = masks[:pred_t+1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return masks, pred_boxes, pred_class


def random_colour_masks(image):
  colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  r[image == 1], g[image == 1], b[image == 1] = colours[random.randrange(0,10)]
  coloured_mask = np.stack([r, g, b], axis=2)
  return coloured_mask

def instance_segmentation_api(img_path, threshold=0.5, rect_th=3, text_size=3, text_th=3):
  masks, boxes, pred_cls = get_prediction(img_path, threshold)
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  for i in range(len(masks)):
    rgb_mask = random_colour_masks(masks[i])
    img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
    cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th)
    cv2.putText(img,pred_cls[i], boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
  plt.figure(figsize=(20,30))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

instance_segmentation_api(image_path)


### BTW how to read all sys info on the machine

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("CPU")
!cat /proc/cpuinfo
print("RAM")
!cat /proc/meminfo

print("GPU?")
tf.test.gpu_device_name()
device_lib.list_local_devices()

In [16]:
import os
root = os.getcwd()
path = os.path.join(root, "images", "test")
imgs = [x for x in sorted(os.listdir(path)) if x.endswith(".jpg")]
print("list", len(imgs))
print("folder", len(list(os.listdir(path))))

list 36
folder 37


In [ ]:
# Creating the dataset
import os
import numpy as np
import csv
import torch
from PIL import Image


class EgoHandsDataset(torch.utils.data.Dataset):
  def __init__(self, root, transforms, test = False):
    self.root = root
    self.transforms = transforms
    # load all image files, sorting them to
    # ensure that they are aligned
    if test:
      path = os.path.join(root, "images", "test")
    else:
      path = os.path.join(root, "images", "train")

    self.imgs = [x for x in sorted(os.listdir(path)) if x.endswith(".jpg")]
    mask_file_path = [x for x in sorted(os.listdir(path)) if x.endswith(".csv")][0]
    self.mask_entries = []
    with open(self.mask_file_path) as mask_file:
      mask_csv = csv.reader(mask_file)
      for line in mask_csv:
        self.mask_entries.append([line[0], line[4], line[5], line[6], line[7]])

  def __getitem__(self, idx):
    # load image
    img_path = os.path.join(self.root, "images", "test", self.imgs[idx])
    img = Image.open(img_path).convert("RGB")

    # read the entries for the bounding boxes for this image
    image_filename = "images/test/" + self.imgs[idx]        
    entries = [entry for entry in self.mask_entries if entry[0] == image_filename]

    # get bounding box coords
    num_objs = len(entries)
    boxes = []
    for entry in entries:
      boxes.append([entry[1], entry[2], entry[3], entry[4]])

    # convert everything into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    # there is only one class
    labels = torch.ones((num_objs,), dtype=torch.int64)
    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd

    if self.transforms is not None:
      img, target = self.transforms(img, target)

    return img, target

  def __len__(self):
    return len(self.imgs)

class HandOverFaceDataset(torch.utils.data.Dataset):
  def __init__(self, root, transforms):
    self.root = root
    self.transforms = transforms
    # load all image files, sorting them to
    # ensure that they are aligned
    self.imgs = list(sorted(os.listdir(os.path.join(root, "hand_over_face", "images_resized"))))
    self.masks = list(sorted(os.listdir(os.path.join(root, "hand_over_face", "masks"))))

  def __getitem__(self, idx):
    # load images and masks
    img_path = os.path.join(self.root, "hand_over_face", "images_resized", self.imgs[idx])
    mask_path = os.path.join(self.root, "hand_over_face", "masks", self.masks[idx])
    img = Image.open(img_path).convert("RGB")
    # note that we haven't converted the mask to RGB,
    # because each color corresponds to a different instance
    # with 0 being background

    ## YEAH that's not true here, we need to count and separate the masks somehow
    ## the annotations have some info, maybe we can use those??
    mask = Image.open(mask_path)
    # convert the PIL Image into a numpy array
    mask = np.array(mask)
    # instances are encoded as different colors
    obj_ids = np.unique(mask)
    # first id is the background, so remove it
    obj_ids = obj_ids[1:]

    # split the color-encoded mask into a set
    # of binary masks
    masks = mask == obj_ids[:, None, None]

    # get bounding box coordinates for each mask
    num_objs = len(obj_ids)
    boxes = []
    for i in range(num_objs):
        pos = np.where(masks[i])
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])
        boxes.append([xmin, ymin, xmax, ymax])

    # convert everything into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    # there is only one class
    labels = torch.ones((num_objs,), dtype=torch.int64)
    masks = torch.as_tensor(masks, dtype=torch.uint8)

    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["masks"] = masks
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd

    if self.transforms is not None:
        img, target = self.transforms(img, target)

    return img, target

  def __len__(self):
    return len(self.imgs)